In [20]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
import math
import lightning as L
from lightning import Trainer
from torchinfo import summary
import re

In [17]:

with open("training.txt", "r") as f:
    data = f.read().splitlines()
    data = [sentence.strip() for sentence in data]
    data = [sentence for sentence in data if len(sentence)>0]
    escape_character = re.escape(',?;.:/*!+-()[]{}"\'&')
    data = [re.sub(f"[{escape_character}]"," \g<0> ",sentence) for sentence in data]

In [27]:
from collections import Counter
vocab_size = 40000
words = [word for sentence in data for word in sentence.split(" ")]
count = Counter(words)
vocab = [word[0] for word in count.most_common(vocab_size)]
with open("vocab.txt", "w") as f:
    f.write("\n".join(vocab))

In [ ]:
class SentenceDataset(Dataset):
    def __init__(self, sentences):
        super().__init__()
        self.sentences= sentences
    def __getitem__(self, index):
        pass
    def __len__(self):
        return len(self.sentences)